# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents import ProjectCreatedEvent
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x146c67980>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x1440d4290>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x1440d4d40>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x1440d4290>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x1440d4d40>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 10 events
"Project Created" occurred at 2024-04-29T23:20:03.694018+02:00
"Project Created" occurred at 2024-04-29T23:20:03.700241+02:00
"Project Created" occurred at 2024-04-29T23:20:03.703940+02:00
"Project Created" occurred at 2024-04-29T23:20:03.708601+02:00
"Project Created" occurred at 2024-04-29T23:20:03.712098+02:00
"Project Created" occurred at 2024-04-29T23:20:03.716089+02:00
"Project Created" occurred at 2024-04-29T23:20:03.719572+02:00
"Project Created" occurred at 2024-04-29T23:20:03.722309+02:00
"Project Created" occurred at 2024-04-29T23:20:03.725729+02:00
"Project Created" occurred at 2024-04-29T23:20:03.728553+02:00
Found 97 project summaries
Project 83195f32-5944-407b-9ff6-1b70888a9d64: Rio Primero
Project 8b0eb583-8ef4-4925-8bd5-c2876bb2f821: Rio Primero
Project 14fd5e74-6bb0-4c37-acf9-7ea95340dc45: Rio Primero
Project 5c8e2cab-4f0d-414a-ae04-5b8770c4d093: Rio Primero
Project f672cab7-7de8-4f52-9771-75ec3f3bd17f: Rio Primero
Project fc01b724-20c7-47f9-bc77-38e9c

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 10 events
Reset projection
Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 10 project summaries
Project efaf8ec3-cebe-4084-9109-2a0f1f345a57: Project 0
Project 0803f4c3-e105-4618-bc00-05f2c5f266ae: Project 1
Project aa78c64e-b559-4c25-8a94-ea00770a2a6a: Project 2
Project d1b9316a-7595-4959-8e74-98a145e5d919: Project 3
Project 21590a2b-4765-4069-bb24-50fe4adbe2eb: Project 4
Project 7aa7e9ba-52a2-4ab7-ae47-d463fe58ceec: Project 5
Project d79ca0d8-1c80-4128-adf1-0ec725f8b007: Project 6
Project dd2f368c-b5d7-4f2c-bca1-3be59c001222: Project 7
Project df4d51b4-854e-44e0-9907-033f7d87369e: Project 8
Project 0e7e33a1-6d64-4341-a7cf-25330af0426f: Project 9
